In [1]:
import json

def getRawPageDataFromFile(file):
    with open(file, 'r') as data_file:
        data = json.load(data_file)

    return data#[:10080]

In [2]:
from SoupArticle import *

def process_page(page):
    url=page["url"]

    soup_article = SoupArticle(page["body"])

    title = soup_article.getTitle()
    location = soup_article.getLocation()
    date= soup_article.getDate()
    nr = soup_article.getNr()
    text = soup_article.getText()

    details = {"url":url, "title":title, "location":location, "date":date, "nr":nr, "text":text}

    return details

ModuleNotFoundError: No module named 'SoupArticle'

In [ ]:
data = getRawPageDataFromFile(file="data.json")

# Build a list of all words

In [ ]:
details=[]

all_words=[]

for p in data:
    page_details = process_page(page=p)
    
    details.append(page_details)
    
    try:
        split_text = page_details["text"].replace(".", " ").replace(","," ").split(" ")
        
        for s in split_text:
            all_words.append(s.lower())
        
    except Exception as e:
        print("Error on document:", page_details)
        print(format(e))
              
all_words=list(set(all_words))

In [ ]:
print(len(all_words))
all_words[:40]

# Build a list for each length of a word

In [ ]:
import numpy as np

four_five = []
six_seven = []
eight_nine = []
ten_eleven = []
twelve_thirteen = []
more = []

"""max_word_length = 0
for w in all_words:
    if len(w)>max_word_length:
        max_word_length=len(w)"""

word_len_matrix = {}#[None] * max_word_length

def build_lists(d):
    for w in d:
        """if len(w)==4 or len(w)==5:
            four_five.append[]"""
        """if len(word_len_matrix)+1<len(w):
            word_len_matrix.append([])
            word_len_matrix[]"""
        #word_len_matrix[len(w)].append(w)
        try:
            values = word_len_matrix[len(w)]
            values.append(w)
            #print(values)
        except:
            values = [w]
        
        new_dict_part = {len(w):values}
        word_len_matrix.update(new_dict_part)

build_lists(all_words)

In [ ]:
#word_len_matrix[5]

# Lookup distances in the lists

In [ ]:
from pyxdameraulevenshtein import damerau_levenshtein_distance, normalized_damerau_levenshtein_distance
import textdistance
import datetime
import time

#TODO: look up the near rows too
#bei Vergleichen mit laengeren, kuerzeren Woertern, darf die Distanz hoch sein, 
#beim Vergleich mit gleichlangen Woertern eher nicht
#bei kuerzeren Woertern, sollte die Distanz generell nur sehr niedrig sein

#looks up, which near rows can even exist and returns a list of the valid ones
#n - how near
#n_letters - to which length
#longest_word - don't get higher than this
def get_near_rows(n, n_letters, possible_indices):
    res = []
    offset_near_rows = list(range(n_letters-n, n_letters+n+1))
    #offset_near_rows = list(filter(lambda x: x>=0 and x<=longest_word, offset_near_rows))
    offset_near_rows = np.intersect1d(offset_near_rows, possible_indices)
    
    return offset_near_rows
        

#allowed_distance should be at least equal to n_near_rows
def lookup(input_string, allowed_distance, n_results, lookup_list, dist_func, n_near_rows=2):
    n_letters = len(input_string)
    
    #contains the row indices, that are relevant for the lookup
    i_lookup_rows = get_near_rows(n_near_rows, n_letters, list(lookup_list.keys()))
    
    #get the relevant rows for the lookup #+ their distance regarding the length of the words
    lookup_rows = []
    for i in i_lookup_rows:
        row_index_dist = abs(n_letters-i)
        #lookup_rows.append((row_index_dist, lookup_list[i]))
        lookup_rows.append(lookup_list[i])
    
    #determine the distance between the given word and every word from the relevant rows
    distances_with_words = []
    for lookup_row in lookup_rows:
        #for w in lookup_row"""[1]""":
        for w in lookup_row:
            dist = dist_func(w, input_string)
            #if dist <= allowed_distance+lookup_row[0]:
            """if dist <= allowed_distance:"""
            distances_with_words.append((dist, w))
            
    #return distances_with_words
    
    dtype = [('distance', float), ('word', object)]
    distances = np.array(distances_with_words, dtype)
    sorted_distances_with_words = np.sort(distances, order='distance')
    
    if len(sorted_distances_with_words)<=n_results:
        return list(sorted_distances_with_words)
    return list(sorted_distances_with_words[:n_results])

#allowed_distance should be at least equal to n_near_rows
def lookup_jaro(input_string, min_sim, n_results, lookup_list, n_near_rows=2, prefix_w=0.1):
    n_letters = len(input_string)
    
    #contains the row indices, that are relevant for the lookup
    i_lookup_rows = get_near_rows(n_near_rows, n_letters, list(lookup_list.keys()))
    
    #get the relevant rows for the lookup + their distance regarding the length of the words
    lookup_rows = []
    for i in i_lookup_rows:
        row_index_dist = abs(n_letters-i)
        #lookup_rows.append((row_index_dist, lookup_list[i]))
        lookup_rows.append(lookup_list[i])
    
    #determine the distance between the given word and every word from the relevant rows
    distances_with_words = []
    #index=0
    for lookup_row in lookup_rows:
        #for w in lookup_row"""[1]""":
        for w in lookup_row:
            dist = textdistance.JaroWinkler()(w, input_string, prefix_weight=prefix_w)
            #if dist <= allowed_distance+lookup_row[0]:
            """if min_sim <= dist:"""
            distances_with_words.append((dist, w))
            
    #print(distances_with_words)
    dtype = [('distance', float), ('word', object)]
    distances = np.array(distances_with_words, dtype)
    sorted_distances_with_words = np.sort(distances, order='distance')
    
    if len(sorted_distances_with_words)<=n_results:
        return list(np.flipud(sorted_distances_with_words))
    return list(np.flipud(sorted_distances_with_words[len(sorted_distances_with_words)-n_results:]))

In [ ]:
def compare_lookup_time(f1, f2):
    t1 = datetime.datetime.now()
    res=f1()
    t2 = datetime.datetime.now()
    d = (t2-t1).total_seconds()*1000
    print("f1 took",d,"milliseconds")
    t1 = datetime.datetime.now()
    res=f2()
    t2 = datetime.datetime.now()
    d = (t2-t1).total_seconds()*1000
    print("f2 took",d,"milliseconds")
    
def l1():
    #lookup(("übarfalasd").lower(), 5, 8, word_len_matrix, dist_func=damerau_levenshtein_distance, n_near_rows=2) 
    lookup(("übarfalasd").lower(), 5, 8, word_len_matrix, dist_func=normalized_damerau_levenshtein_distance, n_near_rows=2)     
    #lookup_jaro("Drohgen".lower(), min_sim=0.8, n_results=3, lookup_list=word_len_matrix, n_near_rows=20, prefix_w=0.2)

def l2():
    #lookup(("übarfalasd").lower(), 5, 10, word_len_matrix,  dist_func=damerau_levenshtein_distance, n_near_rows=20)
    lookup(("übarfalasd").lower(), 5, 8, word_len_matrix, dist_func=normalized_damerau_levenshtein_distance, n_near_rows=20)     
    #lookup_jaro("Drohgen".lower(), min_sim=0.8, n_results=3, lookup_list=word_len_matrix, n_near_rows=2, prefix_w=0.2)
    
compare_lookup_time(l1, l2)

def compare(input_s, top):
    #print("\nLevenshtein:\n", lookup(input_s.lower(), 3, n_results=top, lookup_list=word_len_matrix,  dist_func=textdistance.damerau_levenshtein.distance, n_near_rows=2))
    #print("\nLevenshtein:\n", lookup(input_s.lower(), 3, n_results=top, lookup_list=word_len_matrix,  dist_func=normalized_damerau_levenshtein_distance, n_near_rows=2))  
    print("\nLevenshtein:\n", lookup(input_s.lower(), 3, n_results=top, lookup_list=word_len_matrix,  dist_func=normalized_damerau_levenshtein_distance, n_near_rows=2))      
    print("Jaro Winkler:\n", lookup_jaro(input_s.lower(), min_sim=0.8, n_results=top, lookup_list=word_len_matrix, n_near_rows=2, prefix_w=0.2))

#lookup(("übarfa").lower(), 3, 3, word_len_matrix,  dist_func=damerau_levenshtein_distance, n_near_rows=2)

compare("überfaö", 3)#überfall
compare("übefaö", 3)
compare("übarfa", 3)
compare("Drohgendil", 3)
compare("Drogn", 3)
compare("trogen", 3)
compare("Polzei", 3)
compare("Poliezei", 3)
compare("Slägerei", 3)
compare("Shlaegerei", 3)
compare("Neukoeln", 3)
compare("Nuekoeln", 3)
compare("Fahead", 3)
compare("geklaur", 3)
compare("gestoln", 3)
compare("fahrzzz", 10)

#lookup(("überfaö").lower(), 3, 3, word_len_matrix,  dist_func=textdistance.damerau_levenshtein.distance, n_near_rows=2)
#lookup_jaro(("überfaö").lower(), min_sim=0.8, n_results=3, lookup_list=word_len_matrix, n_near_rows=2, prefix_w=0.2)

#lookup(("übarfa").lower(), 1, None, word_len_matrix,  dist_func=textdistance.jaro_winkler.distance, n_near_rows=0)

In [ ]:
"geklaut" in all_words

In [ ]:
print(textdistance.JaroWinkler()("fahrend", "Fahead".lower()))
print(textdistance.JaroWinkler()("fahrrad", "Fahead".lower()))

In [ ]:
print(damerau_levenshtein_distance("fahrxzy", "fahrrad"))
print(damerau_levenshtein_distance("übarfa", "überhaft"))

In [ ]:
"S".lower()